In [102]:
# Required Python Machine learning Packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
# For preprocessing the data
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
# To split the dataset into train and test datasets
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import GaussianNB
# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score

In [103]:
#load and merge data

df1 = pd.read_csv('historical_data1_Q42008.txt', low_memory=False, sep="|", header=None, \
                  names=['creditScore', 'FirstPayment', 'FirstTimeHomebuyer', 'Maturity', 'MetropolitanArea', 'MorgageInsurancePercentage', 'HouseUnits', 'OccupancyStatus', 'CombinedLoanToValue','DebtToIncome', 'UPB', 'OriginalLoanToValue', 'OrigninalInterestRate', 'Channel', 'PrepaymentPenaltyMorgageFlag', 'ProductType', 'PropertyState', 'PropertyType', 'ZipCode', 'LoanSequenceNumber', 'LoanPurpose', 'OriginalLoanTerm', 'NumberOfBorrowers', 'SellerName', 'ServiceName', 'SuperConformFlag', 'preHARPP'])
                  # dtypes={'creditScore': np.int})
df1.replace('',np.NaN)
df1.replace('NaN',np.NaN)

df2 = pd.read_csv('historical_data1_time_Q42008.txt', low_memory=False, sep="|", header=None, names=['LoanSequenceNumber', 'MonthlyReportingPeriod', 'CurrentActualUPB', 'CurrentLoanDeliquencyStatus', 'LoanAge', 'MonthsToLegalMaturity', 'RepurchaseFlag', 'ModificationFlag', 'ZeroBalanceCode', 'ZeroBalanceEffectiveDate', 'CurrentInterestRate', 'CurrentDefferedUPB', 'DueDateLastOfPaidInstallment', 'MorgageInsuranceRecoveries', 'NetSalesProceeds', 'NonMorgageInsuranceRecoveries', 'Expenses', 'LegalCosts', 'MaintenanceAndPreservationCosts','TaxesAndInsurance', 'MiscellaneousExpenses', 'ActualLoss', 'ModificationCosts'])
df2.replace('',np.NaN)
df2.replace('NaN',np.NaN)

# df = pd.merge(df1, df2, how='right', on = 'LoanSequenceNumber')

'''
display(df1.dtypes)
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):
    display(df1, df2)
'''

"\ndisplay(df1.dtypes)\nwith pd.option_context('display.max_rows', 10, 'display.max_columns', None):\n    display(df1, df2)\n"

In [118]:
# Removing columns
df = pd.merge(df1, df2, how='right', on = 'LoanSequenceNumber')
df.drop(df.columns[len(df.columns)-11:len(df.columns)], axis=1, inplace=True)
df.drop(['OccupancyStatus', 'Channel', 'PrepaymentPenaltyMorgageFlag', 'ProductType', 'LoanPurpose', 'SellerName', 'ServiceName', 'SuperConformFlag', 'preHARPP', 'MonthsToLegalMaturity', 'RepurchaseFlag', 'CurrentDefferedUPB'], 1, inplace=True)

'''with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(df)'''

"with pd.option_context('display.max_rows', 100, 'display.max_columns', None):\n    display(df)"

In [119]:
# Get delinquent observations
some_values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']
default = df['CurrentLoanDeliquencyStatus'].isin(some_values)
df['default'] = default
# need to account for the fact that someone who defaults will always have an instance of not defaulting


#with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
#    display(df[pd.notnull(df['ZeroBalanceCode'])])

In [120]:
logdf = abc.loc[:,['default','creditScore', 'DebtToIncome', 'OriginalLoanToValue', 'OrigninalInterestRate']]

'''
, 'DebtToIncome', 'OriginalLoanToValue','OrigninalInterestRate', 'OriginalLoanTerm'
'''
#display(logdf[logdf['default'] == True])
#logdf.drop_duplicates(subset='LoanSequenceNumber', inplace=True, keep='last')
logdf['creditScore'].replace('   ', np.NaN, inplace=True)
logdf['DebtToIncome'].replace('   ', np.NaN, inplace=True)
logdf = logdf.dropna()
logdf['creditScore'] = pd.to_numeric(logdf['creditScore'])
logdf['DebtToIncome']= pd.to_numeric(logdf['DebtToIncome'])
logdf = logdf.reset_index(drop=True)
# a = logdf.loc[df['creditScore'] == isnull]


#display(logdf)

In [121]:
logdf.isnull().sum()

default                  0
creditScore              0
DebtToIncome             0
OriginalLoanToValue      0
OrigninalInterestRate    0
dtype: int64

In [122]:
#Encode the categorical columns
le = preprocessing.LabelEncoder()
default_cat = le.fit_transform(logdf.default)
creditScore_cat = le.fit_transform(logdf.creditScore)
DebtToIncome_cat = le.fit_transform(logdf.DebtToIncome)
OriginalLoanToValue_cat = le.fit_transform(logdf.OriginalLoanToValue)
OrigninalInterestRate_cat = le.fit_transform(logdf.OrigninalInterestRate)

In [123]:
#initialize the encoded categorical columns
logdf['default_cat'] = default_cat
logdf['creditScore_cat'] = creditScore_cat
logdf['DebtToIncome_cat'] = DebtToIncome_cat
logdf['OriginalLoanToValue_cat'] = OriginalLoanToValue_cat
logdf['OrigninalInterestRate_cat'] = OrigninalInterestRate_cat

In [124]:
dummy_fields=['default','creditScore','DebtToIncome','OriginalLoanToValue','OrigninalInterestRate']
logdfnew = logdf.drop(dummy_fields, axis = 1)

In [125]:
logdfnew = logdfnew.reindex_axis(['creditScore_cat','DebtToIncome_cat','OriginalLoanToValue_cat','OrigninalInterestRate_cat','default_cat'], axis= 1)

In [126]:
#Splitting Data Up Into Training and Testing
features = logdfnew.values[:,:4]
target = logdfnew.values[:,4]
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size = 0.33, random_state = 10)

In [127]:
#Build a GaussianNB classifier and predict 
clf = GaussianNB()
clf.fit(features_train, target_train)
target_pred = clf.predict(features_test)

In [128]:
#Features tested on
features_test

array([[260,  49,  49, 229],
       [116,  18,  64, 371],
       [280,  39,  36, 290],
       ..., 
       [148,  42,  69, 429],
       [160,  39,  89, 371],
       [250,  30,  69, 128]])

In [129]:
#Preciction Results
target_pred

array([0, 0, 0, ..., 0, 0, 0])

In [130]:
#Test accuracy of model
accuracy_score(target_test, target_pred, normalize = True)

0.98182167222894945